In [0]:
spark.conf.set( "fs.azure.account.key.azuredlssup07.dfs.core.windows.net",
  "VVJVYkrDcgJhspZ/iCp3U9C6YBgXCHY41gdZs2RUhI7n5FRfCJx64hYObzOkqslHNOUEj+enQMeW+AStabwgSw==")

In [0]:
dbutils.fs.mount(
  source = "wasbs://azuredls07@azuredlssup07.blob.core.windows.net",
  mount_point = "/mnt/AzureAccount",
  extra_configs = {"fs.azure.account.key.azuredlssup07.blob.core.windows.net":"VVJVYkrDcgJhspZ/iCp3U9C6YBgXCHY41gdZs2RUhI7n5FRfCJx64hYObzOkqslHNOUEj+enQMeW+AStabwgSw=="})

In [0]:
dbutils.fs.ls("/mnt/AzureAccount/")

In [0]:
import json

Json_Data_File = open("/dbfs/mnt/AzureAccount/Pyspark_With_Json/dwsample3.json","rb")
Json_Data = json.loads("/mnt/AzureAccount/Pyspark_With_Json/dwsample3.json")
print(Json_Data)

In [0]:
File_Path = 'abfss://azuredls07@azuredlssup07.dfs.core.windows.net/Pyspark_With_Json/dwsample3.json'
Json_DF_Updated = spark.read.format("json").option("multiline",True).load(File_Path)

In [0]:
display(Json_DF_Updated)

In [0]:
from pyspark.sql.functions import explode_outer

Json_Final = Json_DF_Updated.select("address.*","age","firstName","gender","lastname",explode_outer(Json_DF_Updated.phoneNumbers).alias("PhoneNumber_Updated"))
Json_Final.show()

In [0]:
# display(Json_Final)
Json_Final.select("city","postalCode","state","streetAddress","age","firstName","gender","lastname","PhoneNumber_Updated.*").show()


In [0]:
Json_Sample_DF_2 = spark.read.format("json").option("multiline",True).load("/mnt/AzureAccount/Pyspark_With_Json/sample.json")
display(Json_Sample_DF_2)

In [0]:
JSON_TEMP = Json_Sample_DF_2.select("CHECK","COL",explode_outer(Json_Sample_DF_2.DATA).alias("Temp"),"IFAM","KTM")
display(JSON_TEMP)

In [0]:
JSON_TEMP1 = JSON_TEMP.select("CHECK.*","COL","Temp","IFAM","KTM").show()

In [0]:
from pyspark.sql.types import StructType , StructField ,StringType,IntegerType
from pyspark.sql.functions import from_json, col

PhoneNumbers_schema = StructType([
    StructField("Number" , IntegerType()),
    StructField("home" ,StringType())
])

Json_DF_Updated.withColumn("phoneNumbers_updated",from_json(col("phoneNumbers"),PhoneNumbers_schema))
Json_DF_Updated.drop("phoneNumbers")
Json_DF_Updated.show()

In [0]:
dbutils.fs.ls('abfss://azuredls07@azuredlssup07.dfs.core.windows.net/Pyspark_With_Json/dwsample3.json')

In [0]:
import json

# File_Path = 'abfss://azuredls07@azuredlssup07.dfs.core.windows.net/Pyspark_With_Json/dwsample3.json'
# File_Path_Url = "https://azuredlssup07.blob.core.windows.net/azuredls07/Pyspark_With_Json/dwsample3.json"
dbutils.fs.put("/temp/Json_Sample_Data.json",
                "/dbfs/mnt/AzureAccount/Pyspark_With_Json/dwsample3.json", True)
Json_Data = open("/temp/Json_Sample_Data.json", "rb")
Json_data_Dict = json.loads(Json_Data.read())
print(Json_data_Dict)